In [ ]:
import requests
from config import api_key
import time
import pandas as pd
pd.set_option('display.max_columns', None)
from time import sleep
import numpy as np

In [ ]:
from bs4 import BeautifulSoup 

# Extract IMDB via Webscraping HTML

In [ ]:
url2 = 'https://www.imdb.com/chart/bottom'
response = requests.get(url2)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
print(soup)

In [ ]:
scraped_movies = soup.find_all('td', class_='titleColumn')
scraped_movies

In [ ]:
movies = []
for movie in scraped_movies:
    movie = movie.get_text().replace('\n', "")
    movie = movie.strip(" ")
    movies.append(movie)
movies

In [ ]:
scraped_ratings = soup.find_all('td', class_='ratingColumn imdbRating')
scraped_ratings

In [ ]:
ratings = []
for rating in scraped_ratings:
    rating = rating.get_text().replace('\n', '')
    ratings.append(rating)
ratings

In [ ]:
scraped_ids = soup.find_all('td', class_='watchlistColumn')
scraped_ids

In [ ]:
ids = []
for id_html in scraped_ids:
    id_title = id_html.find("div")["data-tconst"]
    ids.append(id_title)
ids

In [ ]:
# movie_DF = pd.DataFrame({'Movies': movies, 
#                          'Movie Rating': ratings,
#                         'Movie ID': ids})

movie_DF = pd.DataFrame({'Movie ID': ids, 
                         'Movies': movies,
                        'Movie Rating': ratings})


In [ ]:
movie_DF

#### Cleaning DataFrame

In [ ]:
split_rank = movie_DF['Movies'].str.split('.', n=1, expand=True).rename(columns={0:"Rank", 1:"Movies"})
split_rank

In [ ]:
split_rank3 =split_rank['Movies'].str.extract('(.*)\((\d{4})\)', expand=False).rename(columns={0:"Movies", 1:"Year"})
split_rank3

In [ ]:
split_rank3["Rank"]=split_rank["Rank"]
split_rank3["Movie Rating"]=movie_DF["Movie Rating"]
split_rank3["Movie ID"]=movie_DF["Movie ID"]

split_rank3

# Extract OMDB via Webscraping API

from pprint import pprint

In [ ]:
url = "http://www.omdbapi.com/?apikey=" + api_key + "&i="


movies = ['tt1213644',
 'tt0270846',
 'tt0060666',
 'tt4458206',
 'tt4009460',
 'tt1316037',
 'tt0804492',
 'tt0317676',
 'tt0417056',
 'tt0362165',
 'tt10888594',
 'tt0808240',
 'tt0799949',
 'tt0249516',
 'tt7431594',
 'tt0421051',
 'tt0339034',
 'tt0096870',
 'tt0369226',
 'tt0185183',
 'tt1098327',
 'tt0299930',
 'tt0118589',
 'tt10280276',
 'tt1073498',
 'tt7668842',
 'tt5697572',
 'tt0466342',
 'tt0118665',
 'tt0089280',
 'tt0785077',
 'tt1883367',
 'tt0093300',
 'tt0383222',
 'tt0105643',
 'tt6038600',
 'tt1572311',
 'tt2467046',
 'tt0058548',
 'tt7255568',
 'tt2574698',
 'tt0116756',
 'tt10886166',
 'tt0246894',
 'tt0120207',
 'tt0810913',
 'tt10350922',
 'tt5690360',
 'tt0327554',
 'tt4877122',
 'tt2403029',
 'tt7608028',
 'tt1666186',
 'tt0462244',
 'tt8366590',
 'tt5013984',
 'tt0400426',
 'tt0118539',
 'tt0804452',
 'tt0115624',
 'tt0110978',
 'tt0329028',
 'tt0065832',
 'tt0295427',
 'tt0795461',
 'tt0295254',
 'tt1411664',
 'tt9820556',
 'tt1517489',
 'tt0103923',
 'tt0110647',
 'tt0120185',
 'tt0110857',
 'tt0275022',
 'tt0368226',
 'tt4667094',
 'tt7059844',
 'tt10481868',
 'tt0119707',
 'tt0372873',
 'tt0085750',
 'tt0095560',
 'tt0118688',
 'tt0897361',
 'tt0424774',
 'tt0450345',
 'tt0094074',
 'tt0432291',
 'tt0190374',
 'tt1714203',
 'tt0094118',
 'tt6439558',
 'tt3138104',
 'tt0158622',
 'tt0811138',
 'tt0891592',
 'tt1185266',
 'tt0460780',
 'tt1530509',
 'tt1231277']


In [ ]:
response = requests.get(url)
print(response.status_code)

In [ ]:
title = []
year = []
imddrate = []
rate = []
genre = []
actor = []
gross = []
director = []


for movie_title in movies:
    
    try: 
        
        print(movie_title)
        full_url = url + movie_title
        response = requests.get(full_url)

        time.sleep(1)
        

        if response.status_code == 200:    
            data = response.json()

            title.append(data["Title"])
            year.append(data["Year"])
            imddrate.append(data["imdbRating"])
            rate.append(data["Rated"])
            genre.append(data["Genre"])
            actor.append(data["Actors"])
            gross.append(data["BoxOffice"])
            director.append(data["Director"])
#             count += 1

    except Exception:
            title.append(np.nan)
            year.append(np.nan)
            imddrate.append(np.nan)
            rate.append(np.nan)
            genre.append(np.nan)
            actor.append(np.nan)
            gross.append(np.nan)
            director.append(np.nan)
#             count += 1
#             print("City not found. Skipping...")

In [ ]:
df = pd.DataFrame()
df["Movies"] = title
df["Movie ID"] = movies
df["IMDB Rating"] = imddrate
df["Genre"] = genre
df["Rated"] = rate
df["Actors"] = actor
df["Directors"] = director
df["Gross"] = gross

df

# Transform/Merge Datasets

In [ ]:
merged = pd.merge(split_rank3,df, on=["Movie ID"],how="outer")
merged

In [ ]:
mask = merged.drop(columns=['Movies_y', 'IMDB Rating'])
mask

### Store DataFrame#1-"Raw Main IMDB/OMDB DataFrame" into CSV

In [ ]:
mask.to_csv('IMDB Worst 100 Movies.csv', index=False)

#### Cleaning Data

In [ ]:
# Renaming Movie column
Rename_df = mask.rename(columns={'Movies_x': 'movies', 'Year': 'year', 'Rank': 'rank', "Movie ID": "movie_id", "Movie Rating": "movie_rating", 'Genre': 'genre', 'Rated': 'rated', 'Actors': 'actors', 'Directors': 'directors', 'Gross': 'gross' })
Rename_df

In [ ]:
# Metadata table
new_df = Rename_df[['movie_id','movies', 'rank', 'movie_rating', 'rated', 'year', 'genre', 'actors', 'directors', 'gross']]
new_df

### Store DataFrame#2-"Clean Main IMDB/OMDB DataFrame" into CSV

In [ ]:
new_df.to_csv('movies.csv', index=False)

#### Creating Genre Table

In [ ]:
genre = new_df['genre'].value_counts()
genre

In [ ]:
genre_df = pd.DataFrame(genre.index)
genre_df

In [ ]:
Rename_genre = genre_df.rename(columns={0: 'genre'})
Rename_genre

### Store DataFrame#3-"Genre" into CSV

In [ ]:
Rename_genre.to_csv('genre.csv', index=False)

#### Creating Directors Table

In [ ]:
director = new_df['directors'].value_counts()
director

In [ ]:
director_df = pd.DataFrame(director.index)
director_df

In [ ]:
Rename_director = director_df.rename(columns={0: 'directors'})
Rename_director

### Store DataFrame#4-"Director" into CSV

In [ ]:
Rename_director.to_csv('director.csv', index=False)

#### Creating Rated Table

In [ ]:
# Rated table
rated = new_df['rated'].value_counts()
rated




In [ ]:
Rated_df = pd.DataFrame(rated.index)
Rated_df

In [ ]:
Rename_rated = Rated_df.rename(columns={0: 'rated'})
Rename_rated

### Store DataFrame#6-"Rated" into CSV

In [ ]:
Rename_rated.to_csv('rated.csv', index=False)

# LOAD

In [ ]:
import sqlalchemy
from sqlalchemy import create_engine

## Connect to cloud database

In [ ]:
rds_connection_string = "postgres:password@35.223.203.48:5432/postgres"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Create Tables

In [ ]:
engine.table_names()

### Loading date from CSV to database

In [ ]:
#genres 
Rename_genre.to_sql(name='lk_genres', con=engine, if_exists='append', index=False, method="multi")

In [ ]:
#rated
Rename_rated.to_sql(name='lk_rated', con=engine, if_exists='append', index=False, method="multi")

In [ ]:
#director
Rename_director.to_sql(name='lk_directors', con=engine, if_exists='append', index=False, method="multi")

# Queries

In [ ]:
genres_df = pd.read_sql("Select * from lk_genres", con=engine)
genres_df.head()

In [ ]:
rated_df = pd.read_sql("Select * from lk_rated", con=engine)
rated_df.head()

In [ ]:
directors_df = pd.read_sql("Select * from lk_directors", con=engine)
directors_df.head()

In [ ]:
# merging by movie table with genre table by genre_id
merge_genre = pd.merge(new_df, genres_df, on="genre")
merge_genre

In [ ]:
# merging by movie table by rated
merge_rated =  pd.merge(merge_genre, rated_df, on="rated")
merge_rated

In [ ]:
# merging by movie table by director
merge_final =  pd.merge(merge_rated, directors_df, on="director")
merge_final

In [ ]:
final_df = merge_final.drop(["last_updated_x", "last_updated_y", "last_updated", "genre", "director", "rated"], axis=1)
final_df.head()

In [ ]:
#Use pandas to load movies csv into converted DataFrame into database
final_df.to_sql(name='movies', con=engine, if_exists='append', index=False, method="multi")

In [ ]:
movies_df = pd.read_sql("Select * from movies", con=engine)
movies_df.head()

# Bonus

In [ ]:
# matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Design a query to count the total number per 'Rated'


query = """
            SELECT
                rated,
                avg(movie_rating) as avg_rating,
               count(rated) as rated_count
            FROM
                movies m
            JOIN lk_rated lr on m.rated_id = lr.rated_id
            GROUP BY
                rated
            order by
                rated_count asc;
        """

# Query All Records in the the Database
rated_count = pd.read_sql(query, con=engine)

rated_count.head()

In [ ]:
plt.figure(figsize=(10,6))
sns.barplot(data=rated_count, x="rated", y="rated_count")
plt.title("Movie Rating")
plt.show()

In [ ]:
data2 = rated_count.sort_values(by="avg_rating")

plt.figure(figsize=(10,6))
sns.barplot(data=data2, x="rated", y="avg_rating")
plt.title("Average Movie Rating")
plt.show()